In [1]:
'''handle Imbalancing Classes'''

import pandas as pd
import re
import math
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE   #!!!!!
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
       
def preprocess(df, class_index, n): 
    class_DFdict = dict()
    docs_TF = []
    
    for index, row in df.iloc[class_index].iterrows():
        text = str(row["title"]) + str(row["content"])  
        text = re.sub(r'[^\w]','', text)  #remove string not unicode decoded
        text = re.sub(r"[A-Za-z0-9]", "", text) #remove english & number

        doc_TFdict = dict()
        doc_has_word = []
        for t in range(len(text)-(n-1)):
            words = text[t:t+n]  
            # tf, df
            #if words not in stopword:
            if words in doc_TFdict:      #term freq of that class
                doc_TFdict[words] += 1
            else:
                doc_TFdict[words] = 1
            if words not in doc_has_word:
                if words in class_DFdict:     #df
                    class_DFdict[words] += 1
                else:
                    class_DFdict[words] = 1  
                doc_has_word.append(words)
        docs_TF.append(doc_TFdict)      
        
    return docs_TF, class_DFdict

### umc

In [2]:
firm = 'umc' #TAIEX, tsmc, umc, MediaTek
for dataset_index in [4]:  #[3, 4]  #3效果不好~~
    rise_articles = pd.read_csv('keyword_dataset/{}_{}_rise_articles.csv'.format(dataset_index,firm), encoding="UTF-8")[['title','content']]
    rise_keywords = pd.read_csv('keyword_dataset/{}_{}_rise_keywords.csv'.format(dataset_index,firm), encoding="UTF-8")  
    down_articles = pd.read_csv('keyword_dataset/{}_{}_down_articles.csv'.format(dataset_index,firm), encoding="UTF-8")[['title','content']]
    down_keywords = pd.read_csv('keyword_dataset/{}_{}_down_keywords.csv'.format(dataset_index,firm), encoding="UTF-8")
   
    for remove in [False, True]: 
        if remove == True:
            r_kw_list = list(rise_keywords['gram'])
            d_kw_list = list(down_keywords['gram'])
            #print(rise_keywords.shape, down_keywords.shape)

            for index, row in rise_keywords.iloc[:].iterrows():  
                if row['gram'] in d_kw_list:
                    rise_keywords = rise_keywords.drop([index],axis=0) #drop row
            for index, row in down_keywords.iloc[:].iterrows():  
                if row['gram'] in r_kw_list:
                    down_keywords = down_keywords.drop([index],axis=0) #drop row
        #print(rise_keywords.shape, down_keywords.shape)
      
        for topk in  [300,400,500,700]:   #[300,400,500,600,700]
            for topk_index in ['TF_IDF']:   #['TF_IDF','TF','TF*DF_chisquare','MI','Lift'] 

                # ### 選top 500 keywords
                rise_topk_keywords = rise_keywords.sort_values(by=[topk_index])[:topk]['gram']
                down_topk_keywords = rise_keywords.sort_values(by=[topk_index])[:topk]['gram']
                all_topk_keywords = list(rise_topk_keywords)+ list(down_topk_keywords)
               
                # ### 轉成矩陣
                # x: top500 keywords, y: 每篇文章
                # 內容: tf / tf-idf / tf-df*chisquare
                umc_articles = pd.concat([rise_articles,down_articles],axis=0)
                target_doc = list(range(umc_articles.shape[0]))
                target_doc_count = len(target_doc)
                y = np.concatenate((np.ones(rise_articles.shape[0]),np.zeros(down_articles.shape[0])))  #看漲文章:1, 看跌文章:0 

                # tf, df
                TFlist = []  # ngram, each document, (keyword, tf)
                DFlist = []  # ngram, (keyword, df)
                ngrm = [2,3,4,5,6]
                for i in ngrm:
                    docs_TF, class_DF = preprocess(umc_articles, target_doc, i)       
                    TFlist.append(docs_TF)
                    DFlist.append(class_DF)

                # X
                X =  np.zeros((target_doc_count, 2*topk)) 
                for i in range(target_doc_count):    # y: 每篇文章 
                    for j, kw in enumerate(all_topk_keywords): # x: top500 keywords
                        # tf-idf   
                        # tf: docs_TF[i][kw], df: class_DF[kw]  
                        ngrm = len(kw)-2
                        if kw in TFlist[ngrm][i]:
                            X[i][j] = (1+math.log10(TFlist[ngrm][i][kw]))*math.log10(target_doc_count/DFlist[ngrm][kw])      
                
                
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                # normalization!!!   
                xscaler = preprocessing.StandardScaler().fit(X_train)
                X_train = xscaler.transform(X_train)
                X_test = xscaler.transform(X_test)
                
                # resampling!!! 兩個classes的文章數大約為3:2
                X_train, y_train = SMOTE(k_neighbors=7).fit_resample(X_train, y_train)   #for kneibr in range(1,8):效果差不多
                # PCA!!! 效果不好~~
                #X_res = PCA(n_components=800).fit_transform(X_res)
                #X_res = MinMaxScaler().fit_transform(X_res)
  
                # ### training
                '''RandomForestClassifier'''
                n_estimators = [1000]  #[500, 800, 1000, 2000, 5000]
                for n in n_estimators:
                    model = RandomForestClassifier(n_estimators = n)
                    model.fit(X_train, y_train)
                    ypred = model.predict(X_test)
                    
                    print('RF', remove, dataset_index , topk, topk_index, n)
                    print(confusion_matrix(y_test, ypred))
                    print(classification_report(y_test, ypred))

                '''Support Vector Classifier'''
                for gm in ['scale']: #['scale','auto']表現差不多
                    model = SVC(gamma=gm)  #Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’. 'scale':1/(n_features * X.var()), 'auto':1/n_features.
                    model.fit(X_train, y_train)
                    predictions = model.predict(X_test)
    
                    print('SVC', remove, dataset_index , topk, topk_index, gm)
                    print(confusion_matrix(y_test, predictions))
                    print(classification_report(y_test, predictions))
                           
                '''kNN'''
                n_neighbors = [3,5]
                for n in n_neighbors:
                    knn = KNeighborsClassifier(n_neighbors=n) #weights='distance'
                    knn.fit(X_train,y_train)
                    pred = knn.predict(X_test)
    
                    print('kNN', remove, dataset_index , topk, topk_index, n)
                    print(confusion_matrix(y_test, pred))
                    print(classification_report(y_test, pred))

RF False 4 300 TF_IDF 1000
[[190  23]
 [ 28 127]]
              precision    recall  f1-score   support

         0.0       0.87      0.89      0.88       213
         1.0       0.85      0.82      0.83       155

    accuracy                           0.86       368
   macro avg       0.86      0.86      0.86       368
weighted avg       0.86      0.86      0.86       368

SVC False 4 300 TF_IDF scale
[[183  30]
 [ 30 125]]
              precision    recall  f1-score   support

         0.0       0.86      0.86      0.86       213
         1.0       0.81      0.81      0.81       155

    accuracy                           0.84       368
   macro avg       0.83      0.83      0.83       368
weighted avg       0.84      0.84      0.84       368

kNN False 4 300 TF_IDF 3
[[155  58]
 [ 28 127]]
              precision    recall  f1-score   support

         0.0       0.85      0.73      0.78       213
         1.0       0.69      0.82      0.75       155

    accuracy                    

kNN True 4 400 TF_IDF 3
[[182  31]
 [ 49 106]]
              precision    recall  f1-score   support

         0.0       0.79      0.85      0.82       213
         1.0       0.77      0.68      0.73       155

    accuracy                           0.78       368
   macro avg       0.78      0.77      0.77       368
weighted avg       0.78      0.78      0.78       368

kNN True 4 400 TF_IDF 5
[[183  30]
 [ 51 104]]
              precision    recall  f1-score   support

         0.0       0.78      0.86      0.82       213
         1.0       0.78      0.67      0.72       155

    accuracy                           0.78       368
   macro avg       0.78      0.77      0.77       368
weighted avg       0.78      0.78      0.78       368

RF True 4 500 TF_IDF 1000
[[193  20]
 [ 43 112]]
              precision    recall  f1-score   support

         0.0       0.82      0.91      0.86       213
         1.0       0.85      0.72      0.78       155

    accuracy                           

### tsmc

In [5]:
firm = 'tsmc' #TAIEX, tsmc, umc, MediaTek
for dataset_index in [4]:  #[3, 4]  #3效果不好~~
    rise_articles = pd.read_csv('keyword_dataset/{}_{}_rise_articles.csv'.format(dataset_index,firm), encoding="UTF-8")[['title','content']]
    rise_keywords = pd.read_csv('keyword_dataset/{}_{}_rise_keywords.csv'.format(dataset_index,firm), encoding="UTF-8")  
    down_articles = pd.read_csv('keyword_dataset/{}_{}_down_articles.csv'.format(dataset_index,firm), encoding="UTF-8")[['title','content']]
    down_keywords = pd.read_csv('keyword_dataset/{}_{}_down_keywords.csv'.format(dataset_index,firm), encoding="UTF-8")
   
    for remove in [False, True]: 
        if remove == True:
            r_kw_list = list(rise_keywords['gram'])
            d_kw_list = list(down_keywords['gram'])
            #print(rise_keywords.shape, down_keywords.shape)

            for index, row in rise_keywords.iloc[:].iterrows():  
                if row['gram'] in d_kw_list:
                    rise_keywords = rise_keywords.drop([index],axis=0) #drop row
            for index, row in down_keywords.iloc[:].iterrows():  
                if row['gram'] in r_kw_list:
                    down_keywords = down_keywords.drop([index],axis=0) #drop row
        #print(rise_keywords.shape, down_keywords.shape)
      
        for topk in  [300,400,500,700]:   #[300,400,500,600,700]
            for topk_index in ['TF_IDF']:   #['TF_IDF','TF','TF*DF_chisquare','MI','Lift'] 

                # ### 選top 500 keywords
                rise_topk_keywords = rise_keywords.sort_values(by=[topk_index])[:topk]['gram']
                down_topk_keywords = rise_keywords.sort_values(by=[topk_index])[:topk]['gram']
                all_topk_keywords = list(rise_topk_keywords)+ list(down_topk_keywords)
               
                # ### 轉成矩陣
                # x: top500 keywords, y: 每篇文章
                # 內容: tf / tf-idf / tf-df*chisquare
                umc_articles = pd.concat([rise_articles,down_articles],axis=0)
                target_doc = list(range(umc_articles.shape[0]))
                target_doc_count = len(target_doc)
                y = np.concatenate((np.ones(rise_articles.shape[0]),np.zeros(down_articles.shape[0])))  #看漲文章:1, 看跌文章:0 

                # tf, df
                TFlist = []  # ngram, each document, (keyword, tf)
                DFlist = []  # ngram, (keyword, df)
                ngrm = [2,3,4,5,6]
                for i in ngrm:
                    docs_TF, class_DF = preprocess(umc_articles, target_doc, i)       
                    TFlist.append(docs_TF)
                    DFlist.append(class_DF)

                # X
                X =  np.zeros((target_doc_count, 2*topk)) 
                for i in range(target_doc_count):    # y: 每篇文章 
                    for j, kw in enumerate(all_topk_keywords): # x: top500 keywords
                        # tf-idf   
                        # tf: docs_TF[i][kw], df: class_DF[kw]  
                        ngrm = len(kw)-2
                        if kw in TFlist[ngrm][i]:
                            X[i][j] = (1+math.log10(TFlist[ngrm][i][kw]))*math.log10(target_doc_count/DFlist[ngrm][kw])      
                              
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                # normalization!!!   
                xscaler = preprocessing.StandardScaler().fit(X_train)
                X_train = xscaler.transform(X_train)
                X_test = xscaler.transform(X_test)
                
                # resampling!!! 兩個classes的文章數大約為3:2
                X_train, y_train = SMOTE(k_neighbors=7).fit_resample(X_train, y_train)   #for kneibr in range(1,8):效果差不多
                # PCA!!! 效果不好~~
                #X_res = PCA(n_components=800).fit_transform(X_res)
                #X_res = MinMaxScaler().fit_transform(X_res)
                
                
                # ### training
                '''RandomForestClassifier'''
                n_estimators = [500, 1000]  #[500, 800, 1000, 2000, 5000]
                for n in n_estimators:
                    model = RandomForestClassifier(n_estimators = n)
                    model.fit(X_train, y_train)
                    ypred = model.predict(X_test)
                    
                    print('RF', remove, dataset_index , topk, topk_index, n)
                    print(confusion_matrix(y_test, ypred))
                    print(classification_report(y_test, ypred))

                '''Support Vector Classifier'''
                for gm in ['scale']: #['scale','auto']表現差不多
                    model = SVC(gamma=gm)  #Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’. 'scale':1/(n_features * X.var()), 'auto':1/n_features.
                    model.fit(X_train, y_train)
                    predictions = model.predict(X_test)
    
                    print('SVC', remove, dataset_index , topk, topk_index, gm)
                    print(confusion_matrix(y_test, predictions))
                    print(classification_report(y_test, predictions))
                           
                '''kNN'''
                n_neighbors = [5,9]
                for n in n_neighbors:
                    knn = KNeighborsClassifier(n_neighbors=n) #weights='distance'
                    knn.fit(X_train,y_train)
                    pred = knn.predict(X_test)
    
                    print('kNN', remove, dataset_index , topk, topk_index, n)
                    print(confusion_matrix(y_test, pred))
                    print(classification_report(y_test, pred))

RF False 4 300 TF_IDF 500
[[539 121]
 [120 651]]
              precision    recall  f1-score   support

         0.0       0.82      0.82      0.82       660
         1.0       0.84      0.84      0.84       771

    accuracy                           0.83      1431
   macro avg       0.83      0.83      0.83      1431
weighted avg       0.83      0.83      0.83      1431

RF False 4 300 TF_IDF 1000
[[546 114]
 [116 655]]
              precision    recall  f1-score   support

         0.0       0.82      0.83      0.83       660
         1.0       0.85      0.85      0.85       771

    accuracy                           0.84      1431
   macro avg       0.84      0.84      0.84      1431
weighted avg       0.84      0.84      0.84      1431

SVC False 4 300 TF_IDF scale
[[532 128]
 [110 661]]
              precision    recall  f1-score   support

         0.0       0.83      0.81      0.82       660
         1.0       0.84      0.86      0.85       771

    accuracy                   

SVC True 4 300 TF_IDF scale
[[544 116]
 [273 498]]
              precision    recall  f1-score   support

         0.0       0.67      0.82      0.74       660
         1.0       0.81      0.65      0.72       771

    accuracy                           0.73      1431
   macro avg       0.74      0.74      0.73      1431
weighted avg       0.74      0.73      0.73      1431

kNN True 4 300 TF_IDF 5
[[514 146]
 [286 485]]
              precision    recall  f1-score   support

         0.0       0.64      0.78      0.70       660
         1.0       0.77      0.63      0.69       771

    accuracy                           0.70      1431
   macro avg       0.71      0.70      0.70      1431
weighted avg       0.71      0.70      0.70      1431

kNN True 4 300 TF_IDF 9
[[529 131]
 [306 465]]
              precision    recall  f1-score   support

         0.0       0.63      0.80      0.71       660
         1.0       0.78      0.60      0.68       771

    accuracy                         

### MediaTek

In [6]:
firm = 'MediaTek' #TAIEX, tsmc, umc, MediaTek
for dataset_index in [4]:  #[3, 4]  #3效果不好~~
    rise_articles = pd.read_csv('keyword_dataset/{}_{}_rise_articles.csv'.format(dataset_index,firm), encoding="UTF-8")[['title','content']]
    rise_keywords = pd.read_csv('keyword_dataset/{}_{}_rise_keywords.csv'.format(dataset_index,firm), encoding="UTF-8")  
    down_articles = pd.read_csv('keyword_dataset/{}_{}_down_articles.csv'.format(dataset_index,firm), encoding="UTF-8")[['title','content']]
    down_keywords = pd.read_csv('keyword_dataset/{}_{}_down_keywords.csv'.format(dataset_index,firm), encoding="UTF-8")
   
    for remove in [False, True]: 
        if remove == True:
            r_kw_list = list(rise_keywords['gram'])
            d_kw_list = list(down_keywords['gram'])
            #print(rise_keywords.shape, down_keywords.shape)

            for index, row in rise_keywords.iloc[:].iterrows():  
                if row['gram'] in d_kw_list:
                    rise_keywords = rise_keywords.drop([index],axis=0) #drop row
            for index, row in down_keywords.iloc[:].iterrows():  
                if row['gram'] in r_kw_list:
                    down_keywords = down_keywords.drop([index],axis=0) #drop row
        #print(rise_keywords.shape, down_keywords.shape)
      
        for topk in  [300,500,700]:   #[300,400,500,600,700]
            for topk_index in ['TF_IDF']:   #['TF_IDF','TF','TF*DF_chisquare','MI','Lift'] 

                # ### 選top 500 keywords
                rise_topk_keywords = rise_keywords.sort_values(by=[topk_index])[:topk]['gram']
                down_topk_keywords = rise_keywords.sort_values(by=[topk_index])[:topk]['gram']
                all_topk_keywords = list(rise_topk_keywords)+ list(down_topk_keywords)
               
                # ### 轉成矩陣
                # x: top500 keywords, y: 每篇文章
                # 內容: tf / tf-idf / tf-df*chisquare
                umc_articles = pd.concat([rise_articles,down_articles],axis=0)
                target_doc = list(range(umc_articles.shape[0]))
                target_doc_count = len(target_doc)
                y = np.concatenate((np.ones(rise_articles.shape[0]),np.zeros(down_articles.shape[0])))  #看漲文章:1, 看跌文章:0 

                # tf, df
                TFlist = []  # ngram, each document, (keyword, tf)
                DFlist = []  # ngram, (keyword, df)
                ngrm = [2,3,4,5,6]
                for i in ngrm:
                    docs_TF, class_DF = preprocess(umc_articles, target_doc, i)       
                    TFlist.append(docs_TF)
                    DFlist.append(class_DF)

                # X
                X =  np.zeros((target_doc_count, 2*topk)) 
                for i in range(target_doc_count):    # y: 每篇文章 
                    for j, kw in enumerate(all_topk_keywords): # x: top500 keywords
                        # tf-idf   
                        # tf: docs_TF[i][kw], df: class_DF[kw]  
                        ngrm = len(kw)-2
                        if kw in TFlist[ngrm][i]:
                            X[i][j] = (1+math.log10(TFlist[ngrm][i][kw]))*math.log10(target_doc_count/DFlist[ngrm][kw])      
                
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                # normalization!!!   
                xscaler = preprocessing.StandardScaler().fit(X_train)
                X_train = xscaler.transforjm(X_train)
                X_test = xscaler.transform(X_test)
                
                # resampling!!! 兩個classes的文章數大約為3:2
                X_train, y_train = SMOTE(k_neighbors=7).fit_resample(X_train, y_train)   #for kneibr in range(1,8):效果差不多
                # PCA!!! 效果不好~~
                #X_res = PCA(n_components=800).fit_transform(X_res)
                #X_res = MinMaxScaler().fit_transform(X_res)

                # ### training
                '''RandomForestClassifier'''
                n_estimators = [500, 1000]  #[500, 800, 1000, 2000, 5000]
                for n in n_estimators:
                    model = RandomForestClassifier(n_estimators = n)
                    model.fit(X_train, y_train)
                    ypred = model.predict(X_test)
                    
                    print('RF', remove, dataset_index , topk, topk_index, n)
                    print(confusion_matrix(y_test, ypred))
                    print(classification_report(y_test, ypred))

                '''Support Vector Classifier'''
                for gm in ['scale']: #['scale','auto']表現差不多
                    model = SVC(gamma=gm)  #Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’. 'scale':1/(n_features * X.var()), 'auto':1/n_features.
                    model.fit(X_train, y_train)
                    predictions = model.predict(X_test)
    
                    print('SVC', remove, dataset_index , topk, topk_index, gm)
                    print(confusion_matrix(y_test, predictions))
                    print(classification_report(y_test, predictions))
                           
                '''kNN'''
                n_neighbors = [5,9]
                for n in n_neighbors:
                    knn = KNeighborsClassifier(n_neighbors=n) #weights='distance'
                    knn.fit(X_train,y_train)
                    pred = knn.predict(X_test)
    
                    print('kNN', remove, dataset_index , topk, topk_index, n)
                    print(confusion_matrix(y_test, pred))
                    print(classification_report(y_test, pred))

RF False 4 300 TF_IDF 500
[[138  35]
 [ 29 161]]
              precision    recall  f1-score   support

         0.0       0.83      0.80      0.81       173
         1.0       0.82      0.85      0.83       190

    accuracy                           0.82       363
   macro avg       0.82      0.82      0.82       363
weighted avg       0.82      0.82      0.82       363

RF False 4 300 TF_IDF 1000
[[136  37]
 [ 30 160]]
              precision    recall  f1-score   support

         0.0       0.82      0.79      0.80       173
         1.0       0.81      0.84      0.83       190

    accuracy                           0.82       363
   macro avg       0.82      0.81      0.81       363
weighted avg       0.82      0.82      0.82       363

SVC False 4 300 TF_IDF scale
[[133  40]
 [ 31 159]]
              precision    recall  f1-score   support

         0.0       0.81      0.77      0.79       173
         1.0       0.80      0.84      0.82       190

    accuracy                   

SVC True 4 500 TF_IDF scale
[[137  36]
 [ 48 142]]
              precision    recall  f1-score   support

         0.0       0.74      0.79      0.77       173
         1.0       0.80      0.75      0.77       190

    accuracy                           0.77       363
   macro avg       0.77      0.77      0.77       363
weighted avg       0.77      0.77      0.77       363

kNN True 4 500 TF_IDF 5
[[139  34]
 [ 85 105]]
              precision    recall  f1-score   support

         0.0       0.62      0.80      0.70       173
         1.0       0.76      0.55      0.64       190

    accuracy                           0.67       363
   macro avg       0.69      0.68      0.67       363
weighted avg       0.69      0.67      0.67       363

kNN True 4 500 TF_IDF 9
[[151  22]
 [ 97  93]]
              precision    recall  f1-score   support

         0.0       0.61      0.87      0.72       173
         1.0       0.81      0.49      0.61       190

    accuracy                         

### TAIEX

In [7]:
firm = 'TAIEX' #TAIEX, tsmc, umc, MediaTek
for dataset_index in [4]:  #[3, 4]  #3效果不好~~
    rise_articles = pd.read_csv('keyword_dataset/{}_{}_rise_articles.csv'.format(dataset_index,firm), encoding="UTF-8")[['title','content']]
    rise_keywords = pd.read_csv('keyword_dataset/{}_{}_rise_keywords.csv'.format(dataset_index,firm), encoding="UTF-8")  
    down_articles = pd.read_csv('keyword_dataset/{}_{}_down_articles.csv'.format(dataset_index,firm), encoding="UTF-8")[['title','content']]
    down_keywords = pd.read_csv('keyword_dataset/{}_{}_down_keywords.csv'.format(dataset_index,firm), encoding="UTF-8")
   
    for remove in [False, True]: 
        if remove == True:
            r_kw_list = list(rise_keywords['gram'])
            d_kw_list = list(down_keywords['gram'])
            #print(rise_keywords.shape, down_keywords.shape)

            for index, row in rise_keywords.iloc[:].iterrows():  
                if row['gram'] in d_kw_list:
                    rise_keywords = rise_keywords.drop([index],axis=0) #drop row
            for index, row in down_keywords.iloc[:].iterrows():  
                if row['gram'] in r_kw_list:
                    down_keywords = down_keywords.drop([index],axis=0) #drop row
        #print(rise_keywords.shape, down_keywords.shape)
      
        for topk in  [300,500,700]:   #[300,400,500,600,700]
            for topk_index in ['TF_IDF']:   #['TF_IDF','TF','TF*DF_chisquare','MI','Lift'] 

                # ### 選top 500 keywords
                rise_topk_keywords = rise_keywords.sort_values(by=[topk_index])[:topk]['gram']
                down_topk_keywords = rise_keywords.sort_values(by=[topk_index])[:topk]['gram']
                all_topk_keywords = list(rise_topk_keywords)+ list(down_topk_keywords)
               
                # ### 轉成矩陣
                # x: top500 keywords, y: 每篇文章
                # 內容: tf / tf-idf / tf-df*chisquare
                umc_articles = pd.concat([rise_articles,down_articles],axis=0)
                target_doc = list(range(umc_articles.shape[0]))
                target_doc_count = len(target_doc)
                y = np.concatenate((np.ones(rise_articles.shape[0]),np.zeros(down_articles.shape[0])))  #看漲文章:1, 看跌文章:0 

                # tf, df
                TFlist = []  # ngram, each document, (keyword, tf)
                DFlist = []  # ngram, (keyword, df)
                ngrm = [2,3,4,5,6]
                for i in ngrm:
                    docs_TF, class_DF = preprocess(umc_articles, target_doc, i)       
                    TFlist.append(docs_TF)
                    DFlist.append(class_DF)

                # X
                X =  np.zeros((target_doc_count, 2*topk)) 
                for i in range(target_doc_count):    # y: 每篇文章 
                    for j, kw in enumerate(all_topk_keywords): # x: top500 keywords
                        # tf-idf   
                        # tf: docs_TF[i][kw], df: class_DF[kw]  
                        ngrm = len(kw)-2
                        if kw in TFlist[ngrm][i]:
                            X[i][j] = (1+math.log10(TFlist[ngrm][i][kw]))*math.log10(target_doc_count/DFlist[ngrm][kw])      
                
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                # normalization!!!   
                xscaler = preprocessing.StandardScaler().fit(X_train)
                X_train = xscaler.transform(X_train)
                X_test = xscaler.transform(X_test)
                
                # resampling!!! 兩個classes的文章數大約為3:2
                X_train, y_train = SMOTE(k_neighbors=7).fit_resample(X_train, y_train)   #for kneibr in range(1,8):效果差不多
                # PCA!!! 效果不好~~
                #X_res = PCA(n_components=800).fit_transform(X_res)
                #X_res = MinMaxScaler().fit_transform(X_res)

                # ### training
                '''RandomForestClassifier'''
                n_estimators = [500, 1000, 2000]  #[500, 800, 1000, 2000, 5000]
                for n in n_estimators:
                    model = RandomForestClassifier(n_estimators = n)
                    model.fit(X_train, y_train)
                    ypred = model.predict(X_test)
                    
                    print('RF', remove, dataset_index , topk, topk_index, n)
                    print(confusion_matrix(y_test, ypred))
                    print(classification_report(y_test, ypred))

                '''Support Vector Classifier'''
                for gm in ['scale']: #['scale','auto']表現差不多
                    model = SVC(gamma=gm)  #Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’. 'scale':1/(n_features * X.var()), 'auto':1/n_features.
                    model.fit(X_train, y_train)
                    predictions = model.predict(X_test)
    
                    print('SVC', remove, dataset_index , topk, topk_index, gm)
                    print(confusion_matrix(y_test, predictions))
                    print(classification_report(y_test, predictions))
                           
                '''kNN'''
                n_neighbors = [5,9,13]
                for n in n_neighbors:
                    knn = KNeighborsClassifier(n_neighbors=n) #weights='distance'
                    knn.fit(X_train,y_train)
                    pred = knn.predict(X_test)
    
                    print('kNN', remove, dataset_index , topk, topk_index, n)
                    print(confusion_matrix(y_test, pred))
                    print(classification_report(y_test, pred))

RF False 4 300 TF_IDF 500
[[572 128]
 [ 81 818]]
              precision    recall  f1-score   support

         0.0       0.88      0.82      0.85       700
         1.0       0.86      0.91      0.89       899

    accuracy                           0.87      1599
   macro avg       0.87      0.86      0.87      1599
weighted avg       0.87      0.87      0.87      1599

RF False 4 300 TF_IDF 1000
[[577 123]
 [ 85 814]]
              precision    recall  f1-score   support

         0.0       0.87      0.82      0.85       700
         1.0       0.87      0.91      0.89       899

    accuracy                           0.87      1599
   macro avg       0.87      0.86      0.87      1599
weighted avg       0.87      0.87      0.87      1599

RF False 4 300 TF_IDF 2000
[[574 126]
 [ 88 811]]
              precision    recall  f1-score   support

         0.0       0.87      0.82      0.84       700
         1.0       0.87      0.90      0.88       899

    accuracy                     

RF True 4 300 TF_IDF 1000
[[532 168]
 [179 720]]
              precision    recall  f1-score   support

         0.0       0.75      0.76      0.75       700
         1.0       0.81      0.80      0.81       899

    accuracy                           0.78      1599
   macro avg       0.78      0.78      0.78      1599
weighted avg       0.78      0.78      0.78      1599

RF True 4 300 TF_IDF 2000
[[531 169]
 [175 724]]
              precision    recall  f1-score   support

         0.0       0.75      0.76      0.76       700
         1.0       0.81      0.81      0.81       899

    accuracy                           0.78      1599
   macro avg       0.78      0.78      0.78      1599
weighted avg       0.79      0.78      0.78      1599

SVC True 4 300 TF_IDF scale
[[551 149]
 [204 695]]
              precision    recall  f1-score   support

         0.0       0.73      0.79      0.76       700
         1.0       0.82      0.77      0.80       899

    accuracy                     

In [ ]:
'''
###聯電### [no resampling]
'SVCorRForKNN', remove, dataset_index , topk, topk_index)
# dataset_index=3表現都不好
# topk_index有試了['TF_IDF','TF','TF*DF_chisquare','MI','Lift']，只有TF_IDF預測效果好
 
# f1-score (macro avg) 
RF False 4 [400/500] TF_IDF 1000 0.86   
RF False 4 700 TF_IDF 800 0.86 
RF False 4 [300,600] TF_IDF 1000 0.85  
RF True 4 [300/400/500] TF_IDF 1000 0.82

SVC False 4 [300/700] TF_IDF scale 0.82
SVC True 4 [300/all] TF_IDF scale 0.81
SVC False 4 [400/500/600/700] TF_IDF scale 0.80

kNN False 4 500 TF_IDF 3 0.79
kNN True 4 700 TF_IDF 3 0.79
kNN False 4 [300/600] TF_IDF [3,5] 0.78
kNN False 4 [300/400/500] TF_IDF [5,7,9] 0.77


###台積電### [resampling]
'SVCorRForKNN', remove, dataset_index , topk, topk_index)
# topk_index只有試了['TF_IDF]
    
# f1-score (macro avg) 
[resampling] 0.86 
RF False 4 [300/all] TF_IDF [500/all] 0.84
 
SVC False 4 500 TF_IDF [scale/auto] 0.84
SVC False 4 [300/all] TF_IDF [scale/auto] 0.83

kNN False 4 500 TF_IDF 9 0.75
kNN False 4 [300/400/500] TF_IDF [5/all] 0.74


###聯發科### [resampling] [PCA效果不好] 
'SVCorRForKNN', remove, dataset_index , topk, topk_index)
# topk_index只有試了['TF_IDF]
    
# f1-score (macro avg) 
[resampling] 0.84 
RF False 4 500 TF_IDF 1000 0.83
RF False 4 700 TF_IDF [500/2000] 0.83
RF False 4 300 TF_IDF [500/all] 0.82

[resampling] 0.82
SVC False 4 300 TF_IDF scale 0.80
    
kNN False 4 700 TF_IDF 5 0.74    


###大盤### [no resampling]
'SVCorRForKNN', remove, dataset_index , topk, topk_index)
# topk_index只有試了['TF_IDF]
    
# f1-score (macro avg) 
RF False 4 [500/all] TF_IDF 500 0.87    
RF False 4 [300/500/700] TF_IDF [1000/2000] 0.86
    
SVC False 4 700 TF_IDF [scale/auto] 0.87    
SVC False 4 [300/500] TF_IDF [scale/auto] 0.86
    
kNN False 4 [300/500/700] TF_IDF [5/9/13] 0.78
    
-----------------------------------------------------------------
RF: n_estimators[500, 800, 1000, 2000, 5000]設多少好像預測結果差異很小
SVC gamma[scale/auto] 結果差不多
kNN: n_estimators[5,7,9,11,15,21]設多少好像預測結果差異很小
     weighted = 'uniform' or 'distance'結果差不多
'''